<a href="https://colab.research.google.com/github/sir-sauc3/Project2/blob/main/Sos%26RobertoProject2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sos' and Roberto's Project 2
##A Simple Grazing Model

####Objectives
* to develop a Cellular Automata model of grazing
* to experiment with intervention policies, like culls to examine their effect

####Pre-req
* Review Ch. 8 Diffusion and Ch. 11.5: Sugarscape (from textbook On Complexity)
* Complete Labs 7-8 – this provides the framework used for this project

In [56]:
import numpy as np
from dataclasses import dataclass
from pprint import pprint
import random

In [67]:
@dataclass
class Grazing2D:
  kernel = [[]]
  field = np.random.choice(('G','E'), (50,50), p=(0.5, 0.5))
  movementRate : int
  weightLossRate : int
  M : int
  N = random.randint(20,40)
  defaultWeight = 20



In [71]:
test = Grazing2D(1,1,1)
print(test.N)

28
